In [ ]:
# !aws s3 cp segments_cs_gb s3://monlam.ai.stt/stt_pecha_tools/ --recursive

In [1]:
group = "gf"
target_directory = f"segments_cs_{group}"

In [2]:
import os
def getTimeSpan(filename):
    filename = os.path.splitext(os.path.basename(filename))[0]
    try:
        if "_to_" in filename:
            start, end = filename.split("_to_")
            start = start.split("_")[-1]
            end = end.split("_")[0]
            end = float(end)
            start = float(start)
            return (end - start)/1000
        else:
            start, end = filename.split("-")
            start = start.split("_")[-1]
            end = end.split("_")[0]
            end = float(end)
            start = float(start)
            return (end - start)
    except Exception as err:
        print(f"filename is:'{filename}'. Could not parse to get time span.")
        return 0
    
getTimeSpan("STT_AB00094_0057_290646_to_295860.mp3")
getTimeSpan("STT_AB00094_0057_290646_to_295860")

5.214

In [4]:
import os
rows = []
from transformers import pipeline
generator = pipeline(model="openpecha/wav2vec2_run8")

for root, _, files in os.walk(target_directory):
        for file in files:
            source_file_path = os.path.join(root, file)
            # print(f"{file} https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/{file}")
            opt = generator(f"/media/monlamai/SSD/GitHub/split/STT_CS/segments_cs_{group}/{file}")
            inf = opt['text']
            rows.append([file, f"https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/{file}", inf , getTimeSpan(file)])

In [5]:
rows[0]

['STT_CS-D-2014-P-D-B05-3-T-01_0072_617025_to_619336.wav',
 'https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/STT_CS-D-2014-P-D-B05-3-T-01_0072_617025_to_619336.wav',
 'ཨེ་དེ་ནས་ཁོ་ལ་ཨེ་',
 2.311]

In [6]:
import pandas as pd
df = pd.DataFrame(rows, columns =['file_name', 'url', 'inference_transcript', 'audio_duration'])

In [7]:
df.head()

,file_name,url,inference_transcript,audio_duration
0,STT_CS-D-2014-P-D-B05-3-T-01_0072_617025_to_61...,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ཨེ་དེ་ནས་ཁོ་ལ་ཨེ་,2.311
1,STT_CS-X-2016-P-D-B02-2-R-01_0151_1805332_to_1...,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ན་ཝ།དི་ གུ།ཨ་ཁུ་ྦ་པ་། སྦག་པ།,7.441
2,STT_CS-D-2017-P-D-B01-1-R-03_0013_74097_to_808...,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ཤིང་སྡོང་ད་ཁ་སླེབས་རེད་ད་ར། ་སཨ་ནས་དཀོན་མཆོག་ར...,6.756
3,STT_CS-D-2016-P-D-B01-2-R-02_0034_309496_to_31...,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,ཁོ་ད་སྒོར་གྱི་སྡེབ་གི་ཡོད་རེད་ད་རའ། ཨ་ད་བུ་མོ་...,7.867
4,STT_CS-X-2016-P-D-B02-2-R-01_0146_1767141_to_1...,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,གསར་བ་འདི་ནེད་འགྲོ་གེ། ད་དག་ལྟ་དགོ་།,3.942


In [19]:
df.to_csv(f"segments_cs_{group}_inf.csv", index=False)

In [26]:
df[['inference_transcript','url']].iloc[0:10].to_dict()

{'inference_transcript': {0: 'ར་ལབ་རེད་ད། ཨ་ལེ་བྷེ་བྷ་ལབ་རེད་ད། ཨ་ནས་བུ་དེ་ཡོད་བ།',
  1: 'ཏི་ཙི་བཅུ་ལ་ཐུག་སོང་ལབ་རེད་ད། ཨ་ནས་ཤེན་ྨན་དེ་ཡོད་སེམ་གང་ལས་ཤག་བྱས་རེད་ད་རེད་། ལག་པ།',
  2: 'ད་ཁོ་ཀ་ཅི་སམ་བློ་ཁོ་རེད་རཱད།',
  3: 'ཞུག་དགོ་ལབ་རེད་ད་རད། ཨེ་ནས་དེ་རྗེས་བུ་ཉི་མ་ཞོགས་པ་ཡང་།འབུ་གཅིག་གིས་ར། ྲག་རེད་དར། བུ་གཅིག་དང་བུ་ཆུང།',
  4: 'ག།བཅོག་ཅཡི།',
  5: 'ཨེ་ནས་བུ་དེ་ཆུ་འདྲ་པོ་བཏུག། མི་ལ་ལག་པ་བཙོ་ཚང་ནས་ཁོ་ཡག་པོ་དཔེ་ཆགས་རེད་འདུག ཨ་ནས་བུ་གཅིག་ག་ལ་རྔན་པབུ་མོ་གཅིག་ཁོ་།',
  6: 'ྤྲད་བྱས་རད་ད་ར། ཨ་ནས་ཉི་མ་གཅིག་ཨེ་འདྲ་བྱས་བྱས། ཞུངདྲོ་མ་ར་ཟོང་རེ་ད།དཀ་པོ་སྒང་ལ་ཡོད་་མེ།',
  7: 'ཆུ་ཚོད་ག་ཚོད་རེད་ཤག་རད་རེད་ད་ཡོད་བ། ཆོ་ཚོད།',
  8: 'ཨ་ནས་ར། དི།ད་ཁོ་ཉལ་ག་འགྲོ་རེད་ད། ཡུད་གཉིས་ཞིག་ཡོད་རེད་ད། ཉལ་ག་འགྲོ་རེད་ད་ར།',
  9: 'ཐོ་མ་ཡོད་། ང་་།'},
 'url': {0: 'https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/STT_CS-D-2016-P-D-B01-1-R-01_0015_109104_to_115418.wav',
  1: 'https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/STT_CS-D-2016-M-D-B02-2-R-13_0052_384352_to_390288.wav',
  2: 'https://d38pmlk0v88drf.cloudfront.n